In [41]:


from abc import ABC, abstractmethod


class Population(ABC):

    def __init__(self, list_individuals: [], fit_function):
        self.population = list_individuals
        self.fit_function = fit_function

    def mutating(self):
        self.population = [
            self.mutate_individual(individual)
            for individual in self.population
        ]

    def crossing(self):
        cross_results = [
            self.cross_individual(individual)
            for individual in self.population
        ]

        new_population = []
        for cross_result in cross_results:
            new_population += cross_result

        self.population = new_population

    def selection(self, topn: int):
        fitted_individuals = [
            {
                'rate': self.fit_function(individual),
                'individual': individual
            }
            for individual in self.population
        ]

        fitted_individuals.sort(key=lambda individual: individual['rate'], reverse=True)

        return self._get_population([
            rated_individual['individual']
            for rated_individual in fitted_individuals[:topn]
        ])

    @abstractmethod
    def cross_individual(self, individual):
        raise NotImplemented()

    @abstractmethod
    def mutate_individual(self, individual):
        raise NotImplemented()

    @abstractmethod
    def _get_population(self, individuals: []):
        raise NotImplemented()

    def size(self) -> int:
        return len(self.population)

    def print_3d_population(self):
        for index, individual in enumerate(self.population):
            print(f'Индивид {index}')
            for row in individual:
                print(' '.join([str(word) for word in row]))
            print()



In [42]:
class MockPopulation(Population):

    def _get_population(self, individuals: []):
        return MockPopulation(individuals, self.fit_function)

    def cross_individual(self, individual):
        if len(individual) == 1:
            return [individual]
        return [individual, individual[:len(individual) - 1]]

    def mutate_individual(self, individual):
        return individual

In [46]:
from random import randint


class SimplePopulation(Population):

    def _get_population(self, individuals: []):
        return MockPopulation(individuals, self.fit_function)

    def cross_individual(self, individual):
        return [
            other[::2] + individual[1::2]
            for other in self.population
        ]

    def mutate_individual(self, individual):
        if randint(1, 3) == 2:
            return individual[::2] + individual[1::2]
        return individual


In [44]:

class Genetic:
    def __init__(self, population: Population):
        self.population_size = population.size()
        self.population = population

    def next_population(self) -> Population:
        self.population.crossing()
        self.population.mutating()
        self.population = self.population.selection(self.population_size)
        return self.population

In [4]:
from word2vec_models import LibrusecModel
from generators import Word2VecGenerator

base_context = [word for word in 'любовь боль страдания'.split(' ')]
embedding = LibrusecModel()
generator = Word2VecGenerator(embedding, base_context)

In [38]:
adam = generator.generate()
eve = generator.generate()

In [6]:
from meaning_classifier import MeaningClassifierCNNNavec

classifier = MeaningClassifierCNNNavec()


In [47]:
population = SimplePopulation([adam, eve], classifier.classify_paragraph)
print('Первое поколение')
population.print_3d_population()
population.crossing()
print('Второе поколение')
population.print_3d_population()

Первое поколение
Индивид 0
страх холод боль горькая стыд
боль гнев болью грусть ярость стыд
боль печаль губах гнев злость стыд
болью боль печаль злоба стыд

Индивид 1
страх стыд боль некоторую
страх радость нежность страшную
стыд обрел некоторую
стыд ярость жалость прежнюю

Второе поколение
Индивид 0
страх холод боль горькая стыд
боль печаль губах гнев злость стыд
боль гнев болью грусть ярость стыд
болью боль печаль злоба стыд

Индивид 1
страх стыд боль некоторую
стыд обрел некоторую
боль гнев болью грусть ярость стыд
болью боль печаль злоба стыд

Индивид 2
страх холод боль горькая стыд
боль печаль губах гнев злость стыд
страх радость нежность страшную
стыд ярость жалость прежнюю

Индивид 3
страх стыд боль некоторую
стыд обрел некоторую
страх радость нежность страшную
стыд ярость жалость прежнюю



In [48]:
genetic = Genetic(population)

In [49]:
genetic.next_population().print_3d_population()

Индивид 0
страх холод боль горькая стыд
страх радость нежность страшную
стыд обрел некоторую
болью боль печаль злоба стыд

Индивид 1
страх стыд боль некоторую
страх радость нежность страшную
стыд обрел некоторую
болью боль печаль злоба стыд

Индивид 2
страх холод боль горькая стыд
страх радость нежность страшную
боль печаль губах гнев злость стыд
болью боль печаль злоба стыд

Индивид 3
страх стыд боль некоторую
страх радость нежность страшную
боль печаль губах гнев злость стыд
болью боль печаль злоба стыд

